In [ ]:
#Import library
import numpy as np
import random as rm
import pandas as pd
from matplotlib import pyplot as plt, cm
import sklearn as sc
import seaborn as sns
import os
import sys
import json
import re
import math
import csv
from numpy.lib.shape_base import tile
from sklearn.preprocessing import normalize
import string
import re
from collections import defaultdict
from difflib import SequenceMatcher
import markov_clustering as mc
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph
import imageio
from sklearn.metrics import silhouette_score
import Levenshtein
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
from collections import defaultdict

# Reading cleaned data from csv files

In [ ]:
pending_clean = pd.read_csv('../honeypot/cleaned_csv/pending_clean.csv')
ebilling_clean = pd.read_csv('../honeypot/cleaned_csv/ebilling_clean.csv')
csh_clean = pd.read_csv('../honeypot/cleaned_csv/csh_clean.csv')
#pending_clean=pending_clean.drop([	'Unnamed: 0',	'Unnamed: 0.1',	'Unnamed: 0.1.1'], axis=1)
# ebilling_clean=ebilling_clean.drop([	'Unnamed: 0',	'Unnamed: 0.1',	'Unnamed: 0.1.1'], axis=1)
# csh_clean=csh_clean.drop([	'Unnamed: 0',	'Unnamed: 0.1',	'Unnamed: 0.1.1'], axis=1)


In [ ]:
pending_clean = pending_clean.drop(pending_clean[pending_clean['description'] == 'HTTP header'].index)
ebilling_clean = ebilling_clean.drop(ebilling_clean[ebilling_clean['description'] == 'HTTP header'].index)
csh_clean = csh_clean.drop(csh_clean[csh_clean['description'] == 'HTTP header'].index)
pending_clean = pending_clean.drop(pending_clean[pending_clean['description'] == 'network scanner header'].index)
ebilling_clean = ebilling_clean.drop(ebilling_clean[ebilling_clean['description'] == 'network scanner header'].index)
csh_clean = csh_clean.drop(csh_clean[csh_clean['description'] == 'network scanner header'].index)


In [ ]:
len(set(pending_clean['session']))



# Levenshein

In [ ]:
possible_char = "!%&()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_abcdefghijklmnopqrstuvwxyz|~αβγδεζηθικλμνξοπρστυφχψωΘΔΣΩΠΛ"

In [ ]:
df_conversion = pd.read_csv('command_to_description.csv')

In [ ]:
# test_df = pd.read_csv('./Bayes/data/testing_data.csv')

In [ ]:
# test_desc = set()
# for stream in test_df['commands']:
#     test_desc.update(set(eval(stream)))
# test_desc

In [ ]:
functionality_class_dict = dict()

to_be_conv=list(set(df_conversion["Job"]))

for i in range(len(to_be_conv)):
    functionality_class_dict[to_be_conv[i]] = possible_char[i]
print(functionality_class_dict)

In [ ]:
class Honeypot:
    def strip_session(self):
        sessions = list(set(self.df['session']))
        for sess in sessions: # new session (end of the one arrow)
            df_sliced = self.df.loc[self.df["session"] == sess][["description"]]
            df_sliced.index = range(len(df_sliced))
            if len(df_sliced) == 1:
                continue

            list_long_desc = list(df_sliced['description'])
            tmp_str_desc = ""
            for des in list_long_desc:
                tmp_str_desc += functionality_class_dict[des]

            self.functionality_cluster_known.add(tmp_str_desc)
            self.functionality_cluster_known_count_dict[tmp_str_desc] +=1


            self.functionality_cluster_unknown.add(tmp_str_desc)
            self.functionality_cluster_unknown_count_dict[tmp_str_desc] +=1
            self.session_to_sequence[sess]=tmp_str_desc
            self.sequence_to_session[tmp_str_desc].append(sess)

        
        # for i in range(len(test_df)):
        #     stream = eval(test_df['commands'][i])
        #     if len(stream) == 1:
        #         continue
            
        #     tmp_str_desc = ""
        #     for des in stream:
        #         tmp_str_desc += functionality_class_dict[des]

        #     self.functionality_cluster_known.add(tmp_str_desc)
        #     self.functionality_cluster_known_count_dict[tmp_str_desc] +=1


        #     self.functionality_cluster_unknown.add(tmp_str_desc)
        #     self.functionality_cluster_unknown_count_dict[tmp_str_desc] +=1
            
        
            

    def leven_dist_matrix(self, sequences):
        num_sequences = len(sequences)
        L_distance_matrix = np.zeros((num_sequences, num_sequences))

        for i in range(num_sequences):
            for j in range(i + 1, num_sequences):
            
                L_distance = Levenshtein.distance(sequences[i], sequences[j]) / (len(sequences[i]) + len(sequences[j]))
                L_distance_matrix[i, j] = L_distance
                L_distance_matrix[j, i] = L_distance
        return L_distance_matrix
    
    def fk_leaf_label(self, i):
        return self.functionality_cluster_known[i]


    def __init__(self, df, name):
        self.df = df
        self.name =name
        self.session_to_sequence = defaultdict(str)
        self.functionality_cluster_known = set()
        self.functionality_cluster_known_count_dict = defaultdict(int)
        self.sequence_to_session = defaultdict(list)
        self.functionality_cluster_unknown = set()
        self.functionality_cluster_unknown_count_dict = defaultdict(int)

        self.strip_session()
       
        self.functionality_cluster_known = list(self.functionality_cluster_known)
        self.L_functionality_known =  self.leven_dist_matrix(self.functionality_cluster_known)
        
        self.functionality_cluster_unknown = list(self.functionality_cluster_unknown)
        self.L_functionality_unknown =  self.leven_dist_matrix(self.functionality_cluster_unknown)
    

In [ ]:
pending_clean


In [ ]:
temp = pending_clean.append(ebilling_clean, ignore_index=True)
all_clean = temp.append(csh_clean, ignore_index=True)

In [ ]:
pending_honey = Honeypot(pending_clean,"pending")
ebilling_honey = Honeypot(ebilling_clean,"ebilling")
csh_honey = Honeypot(csh_clean,"csh")
all_honey = Honeypot(all_clean, "all")

In [ ]:
all_honey.sequence_to_session

In [ ]:
all_honey.functionality_cluster_known

In [ ]:
def evaluate_clusters(distance_matrix, max_k):
    silhouette_scores = []
    for k in range(2, max_k + 1):
        clustering = AgglomerativeClustering(n_clusters=k, affinity='precomputed', linkage='complete')
        labels = clustering.fit_predict(distance_matrix)
        score = silhouette_score(distance_matrix, labels, metric='precomputed')
        silhouette_scores.append(score)
    return silhouette_scores
          

In [ ]:
def graph_silhouette_score(L_score, title):
    # Evaluate clusters from 2 to 10 and plot
    max_k = min(len(L_score) -1,30)
    silhouette_scores = evaluate_clusters(L_score, max_k)

    # Print the silhouette scores for different values of k
    for k in range(2, max_k + 1):
        print(f"Silhouette score for {k} clusters: {silhouette_scores[k-2]}")

    # Plotting the silhouette scores
    import matplotlib.pyplot as plt

    plt.plot(range(2, max_k + 1), silhouette_scores, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score')
    plt.title(title)
    plt.show()

## Silhouette Score vs Number of clusters

### Pending

In [ ]:
graph_silhouette_score(pending_honey.L_functionality_known,"[Pending] Functionality clutering known attacker")

### Ebilling

In [ ]:

graph_silhouette_score(ebilling_honey.L_functionality_known,"[Ebilling] Functionality clutering known attacker")

In [ ]:
#graph_silhouette_score(all_honey.L_structure_known,"[All] Structure clutering known attacker")
#graph_silhouette_score(all_honey.L_structure_unknown,"[All] Structure clutering unknown ip")
graph_silhouette_score(all_honey.L_functionality_known,"Silhouette Score")
#graph_silhouette_score(all_honey.L_functionality_unknown,"[All] Functionality clutering unknown ip")
#graph_silhouette_score(all_honey.L_abstract_known,"[All] Abstract clutering known attacker")
#graph_silhouette_score(all_honey.L_abstract_unknown,"[All] Abstract clutering unknown ip")

In [ ]:

graph_silhouette_score(csh_honey.L_functionality_known,"[Csh] Functionality clutering known attacker")

In [ ]:
# Apply Agglomerative Clustering: Selected the cluster size to be close to argmax(Silhouette score)
def agglomerative_clustering(L_distance_matrix, n_clusters=15):
    agg_clustering = AgglomerativeClustering(n_clusters=n_clusters,  affinity='precomputed', linkage='complete') #Alternatively linkage could also be 'single'
    labels = agg_clustering.fit_predict(L_distance_matrix)
    #print("Cluster labels- Lavenshtein Agglomarative Clustering:", labels)
    return labels

In [ ]:
def agglomerative_clustering_print(L_distance_matrix):
    agg_clustering = AgglomerativeClustering(n_clusters=15,  affinity='precomputed', linkage='complete') #Alternatively linkage could also be 'single'
    labels = agg_clustering.fit_predict(L_distance_matrix)
    print("Cluster labels- Lavenshtein Agglomarative Clustering:", labels)
    return labels

In [ ]:
def str_cmd_decoding(seq, dic):
    conversion = dict((v,k) for k,v in dic.items())
    ret = []
    print("seq: ", seq)

    for char in seq:
        ret.append(conversion[char])
    return ret



In [ ]:
def dump_csv(name, row1, row2, conv_dict):
    assert len(row1) == len(row2)

    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        field = ['sequence', 'label', 'command']
        writer.writerow(field)
        
        for i in range(len(row1)):
            seq = row1[i]
           
            # if "_s" in name:
            #     tmp = [row1[i], row2[i]]

            #     for i in str_cmd_decoding(str(seq),conv_dict):
            #         tmp.append(label_to_ex_cmd[str(i)])

            #     writer.writerow(tmp )

            # else:
            writer.writerow([row1[i], row2[i], str_cmd_decoding(str(seq),conv_dict)] )



In [ ]:
def dump_csv_count(name, row1, row2, conv_dict, p_count, e_count, c_count, sequence_to_session):
    assert len(row1) == len(row2)

    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        field = ['sequence', 'label', 'command', "pending count", "ebilling count" , "csh count", "sessions"]
        writer.writerow(field)
        print(p_count)
        for i in range(len(row1)):
            seq = str(row1[i])
            print(seq)
            writer.writerow([row1[i], row2[i], str_cmd_decoding(seq,conv_dict), p_count[seq], e_count[seq], c_count[seq], sequence_to_session[seq] ] )



In [ ]:
def dump_clustering(honey, name, k=15):
    if name == "TEST_all_normalized_IP":
        labels = agglomerative_clustering(honey.L_functionality_known,k)

        dump_csv_count(f'{name}_fk_ipip.csv',
                 honey.functionality_cluster_known,
                 labels,
                 functionality_class_dict,
                 pending_honey.functionality_cluster_known_count_dict,
                 ebilling_honey.functionality_cluster_known_count_dict,
                 csh_honey.functionality_cluster_known_count_dict, honey.sequence_to_session)
    else:
        labels = agglomerative_clustering(honey.L_functionality_unknown,k)
        dump_csv(f'{name}_fu.csv',honey.functionality_cluster_unknown, labels, functionality_class_dict)


In [ ]:
all_honey.functionality_cluster_unknown

In [ ]:
dump_clustering(all_honey, "TEST_all_normalized_IP",14)

In [ ]:
all_honey.sequence_to_session["TL?F9KYfg&<Ag"]

In [ ]:
# dump_clustering(pending_honey, "pending_normalized")
# dump_clustering(ebilling_honey, "ebilling_normalized")
dump_clustering(csh_honey, "TEST_csh_normalized")

In [ ]:
import pandas as pd

# Load the two CSV files
file_with_sessions = "./TEST_all_normalized_IP_fk_ipip.csv"  # The file that has 'sessions'
file_without_sessions = "../Bayes/data/training_data.csv"  # The file that needs 'sessions'

df1 = pd.read_csv(file_with_sessions)  # Contains 'command' and 'sessions'
df2 = pd.read_csv(file_without_sessions)  # Contains 'command' but no 'sessions'

# Merge based on 'command' while preserving all rows in df2
df_merged = df2.merge(df1[['command', 'sessions']], on='command', how='left')

# Save the updated file
output_file = "updated_file.csv"
df_merged.to_csv(output_file, index=False)


In [ ]:
#IP address rating

#List of file paths
file_paths = ["ip_report_crm.csv", "ip_report_ebilling.csv", "ip_report_pending.csv"]  # Replace with actual file names

# Initialize an empty dictionary
session_dict = {}

# Read each file and update the dictionary
for file in file_paths:
    df = pd.read_csv(file)
    for _, row in df.iterrows():
        session_dict[row['session']] = {
            'clean': row['clean'],
            'unrated': row['unrated'],
            'suspicious': row['suspicious'],
            'malicious': row['malicious']
        }

# Print or use the combined dictionary
print(session_dict)


In [ ]:
file_paths_test = ["ip_report_crm_testing.csv", "ip_report_ebilling_testing.csv", "ip_report_pending_testing.csv"]  # Replace with actual file names

# Initialize an empty dictionary
session_dict_test = {}

# Read each file and update the dictionary
for file in file_paths_test:
    df = pd.read_csv(file)
    for _, row in df.iterrows():
        session_dict_test[row['session']] = {
            'clean': row['clean'],
            'unrated': row['unrated'],
            'suspicious': row['suspicious'],
            'malicious': row['malicious']
        }

# Print or use the combined dictionary
print(session_dict_test)


In [ ]:
def df_honeypot_rate(file):
# Initialize an empty dictionary
    session_dict = {}

    # Read each file and update the dictionary

    df = pd.read_csv(file)
    for _, row in df.iterrows():
        session_dict[row['session']] = {
            'clean': row['clean'],
            'unrated': row['unrated'],
            'suspicious': row['suspicious'],
            'malicious': row['malicious']
        }
    return df, session_dict

    # Print or use the combined dictionary



In [ ]:
# df_rate_pending = df_honeypot_rate("ip_report_pending.csv")
# df_rate_ebilling = df_honeypot_rate("ip_report_ebilling.csv")
# df_rate_crm = df_honeypot_rate("ip_report_crm.csv")


In [ ]:
# df_rate_pending.to_csv("pending_rating.csv")
# df_rate_ebilling.to_csv("ebilling_rating.csv")
# df_rate_crm.to_csv("crm_rating.csv")

In [ ]:
df=df_merged

# Ensure 'sessions' column is a list (assuming it’s stored as a string of lists in CSV)
df['sessions'] = df['sessions'].apply(lambda x: x.strip("[]").replace("'", "").split(', ') if isinstance(x, str) else [])
df

In [ ]:
df_test = pd.read_csv("../Bayes/data/ground_truth.csv")

# Ensure 'sessions' column is a list (assuming it’s stored as a string of lists in CSV)
df_test['sessions'] = df_test['sessions'].apply(lambda x: x.strip("[]").replace("'", "").split(', ') if isinstance(x, str) else [])
df_test

In [ ]:
def calculate_risk_score(sessions,session_dictionary,t):
    risk_scores = []
    for session in sessions:

        if session in session_dictionary:
            data = session_dictionary[session]

            # print((data['clean'] +data['suspicious']+data['malicious']))
            
            if (data['clean'] +data['suspicious']+data['malicious']) >0:
                if data['malicious'] >t:
                    risk_score = 1
                elif data['suspicious'] + data['malicious']>t:
                    risk_score = 0.5
                else:
                    risk_score = 0

                risk_scores.append(risk_score)

    final_risk_score = sum(risk_scores) / len(risk_scores) if risk_scores else 0
    
    return final_risk_score, len(risk_scores)



In [ ]:

# Function to calculate risk score
def calculate_risk_score(session,session_dictionary,t):
    risk_scores = []


    if session in session_dictionary:
        data = session_dictionary[session]

        # print((data['clean'] +data['suspicious']+data['malicious']))
        
        if (data['clean'] +data['suspicious']+data['malicious']) >0:
            if data['malicious'] >t:
                risk_score = 1
            elif data['suspicious'] + data['malicious']>t:
                risk_score = 0.5
            else:
                risk_score = 0
            #risk_score = (data['clean'] * 0 + (data['suspicious'] * 0.5) + (data['malicious'] * 1)) / (data['clean'] +data['suspicious']+data['malicious'])
            # print(data['clean'],data['suspicious'],data['malicious'],":", risk_score)
            # if risk_score != 0:
            #     risk_score = math.log10(risk_score) * 0.5 + 1
            # if risk_score < 0:
            #     risk_score = 0
            risk_scores.append(risk_score)

    final_risk_score = sum(risk_scores) / len(risk_scores) if risk_scores else 0
    
    return final_risk_score, len(risk_scores)



In [ ]:
import ast

# Safely convert sessions to lists if they are strings
def safe_literal_eval(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return [x]
    return x

# Apply the safe conversion
df['sessions'] = df['sessions'].apply(safe_literal_eval)

# Explode to flatten the sessions
df_flattened = df.explode('sessions').reset_index(drop=True)

# Check result
df_flattened.head()


In [ ]:
type(df_flattened["sessions"][1])

In [ ]:
import ast
df_flattened[['risk_score', 'risk_score_count']] = df_flattened['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,session_dict,1)))
df_flattened

In [ ]:
# Apply function to DataFrame and store both risk score and count

for t in range(10):
    print(f"======================threshold GREATER than {t}======================")
    df[['risk_score', 'risk_score_count']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,session_dict,t)))

    for i in [0,1,3,5,7,8,9]:
        temp = df.loc[df['label'] == i]
        #print(temp['risk_score'])
        weighted_average = sum(temp['risk_score'] * temp['risk_score_count']) / sum(temp['risk_score_count'])
        print(i, weighted_average)

In [ ]:
agreement_threshold = 1

In [ ]:
df[['risk_score', 'risk_score_count']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,session_dict,agreement_threshold)))
train_calc_dict = dict()

for i in [0,1,3,5,7,8,9]:
    temp = df.loc[df['label'] == i]
    #print(temp['risk_score'])
    weighted_average = sum(temp['risk_score'] * temp['risk_score_count']) / sum(temp['risk_score_count'])
    print(i, weighted_average, sum(temp['risk_score_count']))
    
    train_calc_dict[i] = [weighted_average, sum(temp['risk_score_count'])]


In [ ]:
df_test[['risk_score', 'risk_score_count']] = df_test['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,session_dict_test,1)))


In [ ]:
df_test[['risk_score', 'risk_score_count']] = df_test['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,session_dict_test,agreement_threshold)))
df_test.dropna()
df_test.index = range(len(df_test))

test_calc_dict = dict()

for i in [0,1,3,5,7,8,9]:
    if i in list(df_test['ground_truth']):

        temp = df_test.loc[df_test['ground_truth'] == i]

        #print(temp['risk_score'])
        weighted_average = sum(temp['risk_score'] * temp['risk_score_count']) / sum(temp['risk_score_count'])
        print(i, weighted_average, sum(temp['risk_score_count']))
        test_calc_dict[i] = [weighted_average, sum(temp['risk_score_count'])]
        


In [ ]:
for t in range(10):
    print(f"======================threshold GREATER than {t}======================")
    
    df[['risk_score', 'risk_score_count']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,session_dict,t)))
    train_calc_dict = dict()

    for i in [0,1,3,5,7,8,9]:
        temp = df.loc[df['label'] == i]
        #print(temp['risk_score'])
        weighted_average = sum(temp['risk_score'] * temp['risk_score_count']) / sum(temp['risk_score_count'])
        # print(i, weighted_average, sum(temp['risk_score_count']))
        
        train_calc_dict[i] = [weighted_average, sum(temp['risk_score_count'])]
        
    df_test[['risk_score', 'risk_score_count']] = df_test['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,session_dict_test,t)))
    df_test.dropna()
    df_test.index = range(len(df_test))

    test_calc_dict = dict()

    for i in [0,1,3,5,7,8,9]:
        if i in list(df_test['ground_truth']):

            temp = df_test.loc[df_test['ground_truth'] == i]

            #print(temp['risk_score'])
            weighted_average = sum(temp['risk_score'] * temp['risk_score_count']) / sum(temp['risk_score_count'])
            # print(i, weighted_average, sum(temp['risk_score_count']))
            test_calc_dict[i] = [weighted_average, sum(temp['risk_score_count'])]
        else:
            test_calc_dict[i] = [0,0]
            
    for i in [0,1,3,5,7,8,9]:
        score = (train_calc_dict[i][0] *  train_calc_dict[i][1] + test_calc_dict[i][0] *  test_calc_dict[i][1] )/ (train_calc_dict[i][1] +test_calc_dict[i][1])
        print(i, score, train_calc_dict[i][1], test_calc_dict[i][1])

In [ ]:
def honeypot_rate_cluster(file,t):
    _, session_dict = df_honeypot_rate(file) 
    # Load the DataFrame
    df=df_merged

    # Ensure 'sessions' column is a list (assuming it’s stored as a string of lists in CSV)
    df['sessions'] = df['sessions'].apply(lambda x: x.strip("[]").replace("'", "").split(', ') if isinstance(x, str) else [])

    # Function to calculate risk score
    def calculate_risk_score(sessions):
        risk_scores = []
        for session in sessions:
            if session in session_dict:
                data = session_dict[session]
                # print((data['clean'] +data['suspicious']+data['malicious']))
                if (data['clean'] +data['suspicious']+data['malicious']) >0:
                    if data['malicious'] >t:
                        risk_score = 1
                    elif data['suspicious'] + data['malicious']>t:
                        risk_score = 0.5
                    else:
                        risk_score = 0  
                # if (data['clean'] +data['suspicious']+data['malicious']) >0:
                #     risk_score = (data['clean'] * 0 + (data['suspicious'] * 0.5) + (data['malicious'] * 1)) / (data['clean'] +data['suspicious']+data['malicious'])
                #     # print(data['clean'],data['suspicious'],data['malicious'],":", risk_score)
                #     # if risk_score != 0:
                #     #     risk_score = math.log10(risk_score) * 0.5 + 1
                #     # if risk_score < 0:
                #     #     risk_score = 0
                    risk_scores.append(risk_score)

        final_risk_score = sum(risk_scores) / len(risk_scores) if risk_scores else 0

        return final_risk_score, len(risk_scores)

    # Apply function to DataFrame and store both risk score and count
    df[['risk_score', 'risk_score_count']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x)))

    # Display DataFrame
    return df


In [ ]:
def honeypot_rate_cluster(file):
    _, session_dict = df_honeypot_rate(file) 
    # Load the DataFrame
    # df=df_merged

    # Ensure 'sessions' column is a list (assuming it’s stored as a string of lists in CSV)
    # df['sessions'] = df['sessions'].apply(lambda x: x.strip("[]").replace("'", "").split(', ') if isinstance(x, str) else [])
    df= pd.read_csv(file)
    def categorize_row(row):
        if row['malicious'] >= 2:
            return 'malicious'
        elif (row['malicious'] + row['suspicious']) >= 2:
            return 'suspicious'
        else:
            return 'clean'

    # Apply categorization function to each row and add results as a new column
    df['category'] = df.apply(categorize_row, axis=1)

    # Aggregate counts of each category
    category_counts = df['category'].value_counts().reset_index()
    category_counts.columns = ['category', 'count']

    # Output aggregated data to CSV
    category_counts.to_csv(f'aggregated_categories_{file}', index=False)

    # (Optional) print the output to verify
    print(category_counts)

In [ ]:
honeypot_rate_cluster("ip_report_pending.csv")
honeypot_rate_cluster("ip_report_ebilling.csv")
honeypot_rate_cluster("ip_report_crm.csv")
honeypot_rate_cluster("ip_report_pending_testing.csv")
honeypot_rate_cluster("ip_report_ebilling_testing.csv")
honeypot_rate_cluster("ip_report_crm_testing.csv")

In [ ]:
def honeypot_rate_cluster(file):
    _, session_dict = df_honeypot_rate(file) 
    # Load the DataFrame
    df=df_merged

    # Ensure 'sessions' column is a list (assuming it’s stored as a string of lists in CSV)
    df['sessions'] = df['sessions'].apply(lambda x: x.strip("[]").replace("'", "").split(', ') if isinstance(x, str) else [])

    # Function to calculate risk score
    def calculate_risk_score(sessions, rating):
        risk_scores = []
        for session in sessions:
            if session in session_dict:
                data = session_dict[session]
                risk_scores.append(data[rating])
                     
        final_risk_score = sum(risk_scores) 

        return final_risk_score, len(risk_scores)

    # Apply function to DataFrame and store both risk score and count
    df[['clean', '0']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,"clean")))
    df[['unrated', '1']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,"unrated")))
    df[['suspicious', '2']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,"suspicious")))
    df[['malicious', '3']] = df['sessions'].apply(lambda x: pd.Series(calculate_risk_score(x,"malicious")))

    # Display DataFrame
    return df

In [ ]:
# Load the two CSV files
file_with_sessions = "./TEST_all_normalized_IP_fk_ipip.csv"  # The file that has 'sessions'
file_without_sessions = "../Bayes/data/training_data.csv"  # The file that needs 'sessions'

df1 = pd.read_csv(file_with_sessions)  # Contains 'command' and 'sessions'
df2 = pd.read_csv(file_without_sessions)  # Contains 'command' but no 'sessions'

# Merge based on 'command' while preserving all rows in df2
df_merged = df2.merge(df1[['command', 'sessions']], on='command', how='left')

# Save the updated file
output_file = "updated_file.csv"
df_merged.to_csv(output_file, index=False)


df_pending_rated = honeypot_rate_cluster("ip_report_pending.csv")
del df_pending_rated['sessions']
df_pending_rated.to_csv("df_pending_rated_01.csv")



# Load the two CSV files
file_with_sessions = "./TEST_all_normalized_IP_fk_ipip.csv"  # The file that has 'sessions'
file_without_sessions = "../Bayes/data/training_data.csv"  # The file that needs 'sessions'

df1 = pd.read_csv(file_with_sessions)  # Contains 'command' and 'sessions'
df2 = pd.read_csv(file_without_sessions)  # Contains 'command' but no 'sessions'

# Merge based on 'command' while preserving all rows in df2
df_merged = df2.merge(df1[['command', 'sessions']], on='command', how='left')

# Save the updated file
output_file = "updated_file.csv"
df_merged.to_csv(output_file, index=False)



df_ebilling_rated = honeypot_rate_cluster("ip_report_ebilling.csv")
del df_ebilling_rated['sessions']
df_ebilling_rated.to_csv("df_ebilling_rated_01.csv")


# Load the two CSV files
file_with_sessions = "./TEST_all_normalized_IP_fk_ipip.csv"  # The file that has 'sessions'
file_without_sessions = "../Bayes/data/training_data.csv"  # The file that needs 'sessions'

df1 = pd.read_csv(file_with_sessions)  # Contains 'command' and 'sessions'
df2 = pd.read_csv(file_without_sessions)  # Contains 'command' but no 'sessions'

# Merge based on 'command' while preserving all rows in df2
df_merged = df2.merge(df1[['command', 'sessions']], on='command', how='left')

# Save the updated file
output_file = "updated_file.csv"
df_merged.to_csv(output_file, index=False)


df_crm_rated = honeypot_rate_cluster("ip_report_crm.csv")
del df_crm_rated['sessions']
df_crm_rated.to_csv("df_crm_rated_01.csv")

In [ ]:
import pandas as pd

# Create a list to store individual session risk scores
expanded_data = []

# Iterate over each row in the DataFrame
for _, row in df.iterrows():

    label = row['label']  # Extract label
    sessions = row['sessions']  # Extract list of sessions

    for session in sessions:
        if session in session_dict:
            data = session_dict[session]
            total_classifications = data['clean'] + data['suspicious'] + data['malicious']

            if total_classifications > 0:
                risk_score = (data['suspicious'] * 0.5 + data['malicious'] * 1) / total_classifications
                
                # Append to the new dataset (individual risk score per session)
                expanded_data.append({'label': label, 'session': session, 'risk_score': risk_score})

# Convert to a DataFrame
expanded_df = pd.DataFrame(expanded_data)

# Save to CSV
expanded_df.to_csv("session_risk_scores_again.csv", index=False)

# # Display the new DataFrame
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Session Risk Scores CSV", dataframe=expanded_df)
